# MIMIC2 MySQL Browser

This notebook provides a Jupyter interface to the MIMIC2 demo data set. I have tested connecting with both pymysql and mysql.connector. Both of which fail when trying to properly insert parameters into the query string. I've followed the examples on the documentation and it sill just doesn't work. After wasting many, many hours, I'm giving up and just using unsafe string formation.

``cursor.execute("""SELECT title, exam_name, category FROM %s""",(tables[-6][0],))`` 

results in 

``ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ''noteevents'' at line 1")``

In [ ]:
import pymysql
import pandas as pd
import os

In [ ]:
CLASSDIR = os.path.join(os.path.expanduser("~"),"")

In [ ]:
### Connect to MySQL server

Every time the MySQL Docker server comes up it gets a new IP address. use ``docker inspect mimic2-mysql | grep IP`` to identify the correct IP address to use.

In [ ]:
conn = pymysql.connect(host="mysql",port=3306,
                       user="jovyan",passwd='jovyan',
                       db='mimic2')

In [ ]:
cursor = conn.cursor()

In [ ]:
bloodpressure = pd.read_sql("""SELECT * FROM chartevents WHERE 
                            itemid in (6, 51, 455, 6701)""",conn)


In [ ]:
bloodpressure["charttime"][0]

In [ ]:
bloodpressure

In [ ]:
bloodpressure.to_csv("./abp_all.csv")

In [ ]:
#bloodpressure[["value1","value2"]]

In [ ]:
unique_subjects = list(set(bloodpressure['subject_id']))
print(len(unique_subjects))
if not os.path.exists("bp"):
    os.makedirs("bp/subjects")
for s in unique_subjects:
    with open(os.path.join("bp/subjects","%s.txt"%s),"w") as f0:
        f0.write("\n".join(["%s\t%s"%(d[1][0],d[1][1]) for d in bloodpressure[bloodpressure["subject_id"]==s][["value1","value2"]].iterrows()]))

In [ ]:
with open("./systolic.txt","w") as f0:
    bloodpressure.dropna()['VALUE1NUM'][0:1000].as_matrix().tofile(f0,sep=",")

In [ ]:
with open("./diastolic.txt","w") as f0:
    bloodpressure.dropna()['VALUE2NUM'][0:1000].as_matrix().tofile(f0,sep=",")

In [ ]:
#bloodpressure[bloodpressure["subject_id"]==s][["value1","value2"]]

In [ ]:
["%s\t%s"%(d[1][0],d[1][1]) for d in bloodpressure[bloodpressure["subject_id"]==s][["value1","value2"]].iterrows()]